## Interacting with the Batfish service


### Python Imports

In your Python program (or shell) you will need to import Pybatfish modules. 
The most common imports are shown below. Depending your needs, this list may vary.

In [1]:
import pandas as pd
from pybatfish.client.commands import *
from pybatfish.datamodel import *
from pybatfish.datamodel.answer import *
from pybatfish.datamodel.flow import *
from pybatfish.question import *
from pybatfish.question import bfq

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option('display.html.use_mathjax', False)

In [3]:
# Configure all pybatfish loggers to use WARN level
import logging
logging.getLogger('pybatfish').setLevel(logging.WARN)

### Sessions

The Batfish service may be running locally on your machine, or on a remote server. 
The first step to analyzing your configurations is setting up the connection to the Batfish service.

In [4]:
bf_session.host = 'localhost'

### Uploading configurations

Batfish is designed to analyze a series of snapshots of a network.

A *network* is a logical grouping of devices -- it may mean all of the devices in your network, 
or a subset (e.g., all devices in a single datacenter.)
 
A *snapshot* is a state of the network at a given time.
A network may contain many snapshots, allowing you to understand the evolution of your network.

Let's say we will be working with our example datacenter:

In [5]:
bf_set_network('example_dc')

'example_dc'

Now you are ready to create your first snapshot. Batfish can ingest a variety of data in order to model your network,
so let's look at how you can package it as a snapshot.

#### Packaging snapshot data

Batfish expects configuration files to be organized according to a folder structure outlined below. 
The names in bold are keywords and must be used in your directory structure. 
Your snapshot need not have all of these components, 
e.g., if you are not analyzing hosts or iptables configurations, the corresponding folders are not needed.

***

* snapshot _[top-level folder]_
    * **configs** _[folder with configurations for network devices]_
        * router1.cfg
        * router2.cfg
        * ...
    * **hosts** _[folder with host configurations (e.g., pointers to their iptables files)]_
        * host1.json _[see below for the expected format]_
        * host2.json
        * ...
    * **iptables** _[folder with iptables configuration files; host JSON files should point to these files]_
        * host1.iptables
        * host2.iptables
        * ...
    * **batfish** _[supplemental information (not network device configuration) used by Batfish]_
        * isp_config.json _[configuration file used to model Internet Service Providers (ISPs), see below for expected format]_

*** 

See the [example snapshot](https://github.com/batfish/batfish/tree/master/networks/example) in the Batfish repository as an example. 
Note: For explanation convenience, this folder contains some extra files that are not used by Batfish, i.e., `example-network.png`.
You do not need to include diagrams of the network with your snapshot(s).

When you supply the snapshot as a zipped file, the top-level folder (called "snapshot" above) should be part of the zip archive. 

Details on packaging specific vendor data and supplementa batfish information [are described here](../formats.md)

#### Initializing a new snapshot

In [6]:
SNAPSHOT_DIR = '../../networks/example'
bf_init_snapshot(SNAPSHOT_DIR, name='snapshot-2020-01-01', overwrite=True)

'snapshot-2020-01-01'

### Using an existing snapshot

If you have a previously initialized snapshot that you would like to work with, you do not need to re-initialize it.
Simply set the network and snapshot by name:

In [7]:
bf_set_network('example_dc')
bf_set_snapshot('snapshot-2020-01-01')

'snapshot-2020-01-01'

### Loading Questions

Batfish exposes a series of *questions* to users. 
With the help of these questions you can examine data about you network as a whole, 
or individual devices, in a vendor-agnostic way.

To load these questions from the services, run:

In [8]:
load_questions()

### Running Questions

Now that you have initialized (or set) a snapshot, 
you can query the Batfish service to retrieve information about the snapshot.

Note that while Batfish supports a wide variety of devices and configuration constructs, 
it may not fully support your configuration files. 
We always recommend checking the status of the snapshot you just initialized, by runnning `bfq.initIssues`:

In [9]:
bfq.initIssues().answer()

Nodes Source_Lines                       Type  \
0  ['as1border1']  None         Convert warning (redflag)   
1  ['as1border1']  None         Convert warning (redflag)   

                                                         Details Line_Text  \
0  Could not determine update source for BGP neighbor: '3.2.2.2'  None       
1  Could not determine update source for BGP neighbor: '5.6.7.8'  None       

  Parser_Context  
0  None           
1  None

This is the general pattern you will be using for many questions:

* `bfq.<question_name>()` Creates a question (with parameters, if applicable).
* `bfq.<question_name>().answer()` sends a query to Batfish service and returns the results of executing the question
* `bfq.<question_name>().answer().frame()` converts the answer into a [Pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe) for easy data manipulation

These are the very basics of how to interact with the Batfish service.

As a next step, explore [a variety of questions](../questions.rst) that allow you to analyze your network in great detail.